In [3]:
import cuml
from cupy import asnumpy
from joblib import dump, load

In [4]:
import cupy as cp
from cupy import asnumpy 
from cuml.datasets.classification import make_classification
from cuml.preprocessing.model_selection import train_test_split
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.metrics import hamming_loss as sk_hamming_loss

import time


# synthetic dataset dimensions
n_samples = 100000
n_features = 1000
n_classes = 2


# random forest depth and size
n_estimators = 25
max_depth = 10

# generate synthetic data [ binary classification task ]
X, y = make_classification ( n_classes = n_classes,
                             n_features = n_features,
                             n_samples = n_samples,
                             random_state = 0 )

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 0 )

model = cuRF( max_depth = max_depth, 
              n_estimators = n_estimators,
              random_state  = 0 )

trained_RF = model.fit ( X_train, y_train )

predictions = model.predict ( X_test )

def check_length(y_true, y_pred):
    if  len(y_true) == len(y_pred):
        return True
    else:
        return False

def weight_sum(sample_score, sample_weight, normalize):
    if normalize:
        return cp.average(sample_score, weights=sample_weight)
    elif sample_weight is not None:
        return cp.dot(sample_score, sample_weight)
    else:
        return sample_score.sum()

def hamming_loss(y_true, y_pred, *, sample_weight=None):
    if check_length(y_true, y_pred)==True:
        try:
            return  float(weight_sum(y_true != y_pred, sample_weight, normalize=True))
        except:
            raise ValueError("{0} is not supported".format(y_type))

s = time.time()
cu_score = hamming_loss( y_test, predictions, )
e = time.time()
print("Time consumed by cuml: ", e-s)
p = time.time()
sk_score = sk_hamming_loss( asnumpy( y_test ), asnumpy( predictions ) )
d = time.time()
print("Time consumed by sklearn: ", d-p)

print( " cuml Hamming Loss: ", cu_score )
print( " sklearn Hamming Loss : ", sk_score )

<ipython-input-4-1b250a83a01b>:29: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  model = cuRF( max_depth = max_depth,


Time consumed by cuml:  0.0004208087921142578
Time consumed by sklearn:  0.00327301025390625
 cuml Hamming Loss:  0.37576
 sklearn Hamming Loss :  0.37576
